In [1]:
from becquerel import Spectrum
import numpy as np
import importlib
import sys
import matplotlib.pyplot as plt
sys.path.insert(0,r"C:\Users\benhu\Documents\Github\radwatch-analysis")

import analysis_methods as am
import ROI
importlib.reload(am)
importlib.reload(ROI)

filename = 'UCB1901PetraleSole'
c = __import__("UCB1901PetraleSoleConfig")

In [2]:
specname = 'UCB1901PetraleSole' + '.spe'
path = r'C:\Users\benhu\Desktop\Research\DataFiles\fishsamples2019'
spectrum = path + r'\\' + specname
background = r'C:\Users\benhu\Desktop\Research\DataFiles\fishsamples2019\background.spe'
livetime = 173962 #s
wetweight = 519.9 #g
source_energies = [605, 609, 662, 1460, 2614]

params = {
    '605': [1, [[-2, -1], [-0.75, 1.25], [1, 2]]],
    '609': [5, [[-2, -1], [-0.75, 0.75], [1, 2]]],
    '662': [5, [[-2, -1], [-0.4, 0.4], [1, 2]]],
    '1460': [15, [[-2, -1], [-0.5, 0.75], [1, 2]]],
    '2614': [20, [[-2, -1], [-0.4, 0.75], [1, 2]]],
}

spec = Spectrum.from_file(spectrum)
bgspec = Spectrum.from_file(background)
counts = spec.counts_vals
energies = spec.bin_centers_kev

roi = ROI.ROI(spec, bgspec, source_energies, 0)

SpeFile: Reading file C:\Users\benhu\Desktop\Research\DataFiles\fishsamples2019\\UCB1901PetraleSole.spe


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\benhu\\Desktop\\Research\\DataFiles\\fishsamples2019\\\\UCB1901PetraleSole.spe'

In [ ]:
#np.where(spec.bin_centers_kev==605)
mask = (spec.bin_centers_kev > 600)&(spec.bin_centers_kev < 620)

In [ ]:
plt.plot(spec.bin_centers_kev[mask], spec.counts_vals[mask])

In [ ]:
roi.find_peak_energies()
roi.roi_pars

In [ ]:
for key in params:
    roi.set_sideband(int(key), params[key][0], params[key][1])

roi.set_sideband(605, 4, [[-2, -1], [-0.75, 0.5], [2, 4]])
roi.set_sideband(609, 5, [[-2, -1], [-0.75, 0.75], [1, 2]])
roi.set_sideband(662, 5, [[-2, -1], [-0.4, 0.4], [1, 2]])
roi.set_sideband(1460, 15, [[-2, -1], [-0.5, 0.75], [1, 2]])
roi.set_sideband(2614, 20, [[-2, -1], [-0.4, 0.75], [1, 2]])

roi.plot_peak_region(spec, source_energies, 605)
roi.plot_peak_region(spec, source_energies, 609)
roi.plot_peak_region(spec, source_energies, 662)
roi.plot_peak_region(spec, source_energies, 1460)
roi.plot_peak_region(spec, source_energies, 2614)

In [ ]:
roi_counts, roi_unc = roi.get_counts()
print(roi_counts)
print(roi_unc)

eff_func = am.Efficiency()
eff_func.set_parameters()

In [ ]:
efficiency = []
efficiencyunc = []
for i in source_energies:
    efficiency.append(eff_func.get_eff(i))
    efficiencyunc.append(eff_func.get_eff_error(i))

countrate = [i / livetime for i in roi_counts]
uncrate = [i / livetime for i in roi_unc]

print(efficiency)
print(countrate)
print(uncrate)

In [ ]:
sactivity, sactunc = [(i / j) / wetweight for i, j in zip(countrate, efficiency)], [(i / j) / wetweight for i, j in zip(uncrate, efficiency)]
for i, j, k in zip(source_energies, sactivity, sactunc):
    print("Specific activity at", i, "keV:", j, "±", k, "Bq/g (wet weight)")